# Locality Lens - Comprehensive Test Suite

This notebook tests the complete Locality Lens workflow with:
- Basic functionality tests
- Parallel execution verification
- Different user profiles
- Edge cases
- Performance benchmarks
- State inspection

## ⚠️ Important Notes:

1. **Markdown cells** (like this one) are for documentation only - they cannot be executed. Only run Python code cells.
2. **Run cells in order** - Start with Cell 2 (Setup and Imports) first, then run other cells sequentially.
3. **Working directory** - The notebook automatically detects if you're running from `tests/` or project root.
4. **If you get import errors**, make sure you've installed dependencies: `pip install -r requirements.txt`


## Setup and Imports

In [ ]:
import sys
import os
import time
from pathlib import Path

# Add project root to path
# Handle both cases: running from tests/ or from project root
current_dir = Path.cwd()
if current_dir.name == 'tests':
    project_root = current_dir.parent
else:
    project_root = current_dir

sys.path.insert(0, str(project_root))

# Import required modules
try:
    from src.graph.graph import compile_graph
    from src.graph.state import LocalityState
    import json
    from pprint import pprint
    print(f"✅ Imports successful!")
    print(f"📁 Project root: {project_root}")
    print(f"📁 Current dir: {current_dir}")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print(f"📁 Project root: {project_root}")
    print(f"📁 Current dir: {current_dir}")
    print(f"📁 Python path: {sys.path[:3]}")
    raise

✅ Imports successful!


In [6]:
def create_initial_state(
    user_input: str,
    user_profile: str = None
):
    """Create initial state for testing."""
    return {
        "user_input": user_input,
        "user_profile": user_profile,
        "coordinates": None,
        "address": None,
        "osm_data": {},
        "aqi_data": None,
        "selected_metrics": [],
        "statistics": {},
        "user_intent": {},
        "summary": None,
        "recommendations": [],
        "visualization_data": None,
        "errors": [],
        "warnings": [],
        "next_action": "",
        "processing_steps": []
    }


def run_test(graph, state, test_name: str) -> dict:
    """Run a test and return results with timing."""
    print(f"\n{'='*60}")
    print(f"TEST: {test_name}")
    print(f"{'='*60}")
    
    start_time = time.time()
    
    try:
        # Run graph with streaming to track progress
        events = []
        for event in graph.stream(state, stream_mode="updates"):
            events.append(event)
            for node_name, node_state in event.items():
                if isinstance(node_state, dict):
                    steps = node_state.get("processing_steps", [])
                    if steps:
                        print(f"  ✓ {node_name}: {steps[-1]}")
        
        # Get final state
        final_state = graph.invoke(state)
        
        elapsed = time.time() - start_time
        
        return {
            "success": True,
            "elapsed_time": elapsed,
            "final_state": final_state,
            "events": events,
            "errors": final_state.get("errors", []),
            "warnings": final_state.get("warnings", [])
        }
    except Exception as e:
        elapsed = time.time() - start_time
        return {
            "success": False,
            "elapsed_time": elapsed,
            "error": str(e),
            "final_state": None
        }


def print_test_results(result: dict, verbose: bool = False):
    """Print test results in a formatted way."""
    if result["success"]:
        print(f"\n✅ TEST PASSED")
        print(f"⏱️  Time: {result['elapsed_time']:.2f}s")
        
        if result.get("errors"):
            print(f"\n⚠️  Errors: {len(result['errors'])}")
            for error in result["errors"]:
                print(f"   - {error}")
        
        if result.get("warnings"):
            print(f"\n⚠️  Warnings: {len(result['warnings'])}")
            for warning in result["warnings"]:
                print(f"   - {warning}")
        
        final_state = result["final_state"]
        
        if verbose:
            print(f"\n📊 Final State Summary:")
            print(f"   - Coordinates: {final_state.get('coordinates')}")
            print(f"   - Address: {final_state.get('address')}")
            print(f"   - Selected Metrics: {len(final_state.get('selected_metrics', []))}")
            print(f"   - Statistics Count: {len(final_state.get('statistics', {}))}")
            print(f"   - Summary Generated: {final_state.get('summary') is not None}")
            print(f"   - User Intent: {final_state.get('user_intent', {})}")
    else:
        print(f"\n❌ TEST FAILED")
        print(f"⏱️  Time: {result['elapsed_time']:.2f}s")
        print(f"\n💥 Error: {result.get('error')}")


In [7]:
# Compile the graph
print("Compiling graph...")
graph = compile_graph()
print("✅ Graph compiled successfully!")


Compiling graph...
✅ Graph compiled successfully!


## Test 1: Basic Flow - Address Input with Bachelor Profile


In [8]:
test1_state = create_initial_state(
    user_input="Indiranagar, Bangalore",
    user_profile="Bachelor/Young Professional"
)

result1 = run_test(graph, test1_state, "Test 1: Address + Bachelor Profile")
print_test_results(result1, verbose=True)

# Display summary if available
if result1["success"] and result1["final_state"].get("summary"):
    print(f"\n📝 Generated Summary:")
    print("-" * 60)
    summary = result1["final_state"]["summary"]
    print(summary[:500] + "..." if len(summary) > 500 else summary)



TEST: Test 1: Address + Bachelor Profile
  ✓ validate_input: validate_input: SUCCESS - Address detected, needs geocoding
✅ Loaded .env from: /Users/nitish.ranjan/Documents/AiDash/Educational/research/locality-lens/.env
✅ GROQ_API_KEY loaded (length: 56)
  ✓ extract_intent_and_select_metrics: extract_intent_and_select_metrics: SUCCESS - Extracted intent, selected 7 metrics
  ✓ geocode_location: geocode_location: SUCCESS - Geocoded to (12.9732913, 77.6404672)
  ✓ fetch_osm_data: fetch_osm_data: SUCCESS - Fetched 25 POI categories
  ✓ calculate_statistics: calculate_statistics: SUCCESS - Calculated 7 metrics
  ✓ generate_summary: generate_summary: SUCCESS - Summary generated

✅ TEST PASSED
⏱️  Time: 15.26s

⚠️  Warnings: 2
   - Road density calculation not yet implemented
   - Road density calculation not yet implemented

📊 Final State Summary:
   - Coordinates: (12.9732913, 77.6404672)
   - Address: Indiranagar, Jogpalya, Bengaluru Central City Corporation, Bengaluru, Bangalore East, Be

## Test 2: Coordinates Input with Family Profile


In [9]:
test2_state = create_initial_state(
    user_input="12.9716, 77.5946",  # Bangalore coordinates
    user_profile="Family with Kids"
)

result2 = run_test(graph, test2_state, "Test 2: Coordinates + Family Profile")
print_test_results(result2, verbose=True)

# Check selected metrics
if result2["success"]:
    print(f"\n📋 Selected Metrics ({len(result2['final_state'].get('selected_metrics', []))}):")
    for metric in result2["final_state"].get("selected_metrics", [])[:10]:
        print(f"   - {metric}")



TEST: Test 2: Coordinates + Family Profile
  ✓ validate_input: validate_input: SUCCESS - Parsed coordinates (12.9716, 77.5946)
  ✓ extract_intent_and_select_metrics: extract_intent_and_select_metrics: SUCCESS - Extracted intent, selected 8 metrics
  ✓ fetch_osm_data: fetch_osm_data: SUCCESS - Fetched 25 POI categories
  ✓ calculate_statistics: calculate_statistics: SUCCESS - Calculated 8 metrics
  ✓ generate_summary: generate_summary: SUCCESS - Summary generated

✅ TEST PASSED
⏱️  Time: 11.35s

⚠️  Warnings: 2
   - Road density calculation not yet implemented
   - Road density calculation not yet implemented

📊 Final State Summary:
   - Coordinates: (12.9716, 77.5946)
   - Address: None
   - Selected Metrics: 7
   - Statistics Count: 7
   - Summary Generated: True
   - User Intent: {'profile_type': 'family', 'priorities': ['education', 'safety', 'family-friendly amenities'], 'concerns': ['protection', 'quality of life'], 'lifestyle': 'Balanced, family-oriented lifestyle', 'metric_sele

## Test 3: Performance Benchmark - Multiple Locations


In [10]:
test_locations = [
    ("Indiranagar, Bangalore", "Bachelor"),
    ("Koramangala, Bangalore", "Family"),
    ("12.9352, 77.6245", "Student"),  # Another Bangalore location
]

performance_results = []

for location, profile in test_locations:
    test_state = create_initial_state(
        user_input=location,
        user_profile=profile
    )
    
    result = run_test(graph, test_state, f"Performance: {location}")
    
    if result["success"]:
        performance_results.append({
            "location": location,
            "profile": profile,
            "time": result["elapsed_time"],
            "metrics_count": len(result["final_state"].get("selected_metrics", [])),
            "statistics_count": len(result["final_state"].get("statistics", {})),
            "has_summary": result["final_state"].get("summary") is not None
        })
    
    print_test_results(result, verbose=False)

# Summary
print(f"\n{'='*60}")
print("📊 PERFORMANCE SUMMARY")
print(f"{'='*60}")

if performance_results:
    avg_time = sum(r["time"] for r in performance_results) / len(performance_results)
    min_time = min(r["time"] for r in performance_results)
    max_time = max(r["time"] for r in performance_results)
    
    print(f"\n⏱️  Timing:")
    print(f"   - Average: {avg_time:.2f}s")
    print(f"   - Min: {min_time:.2f}s")
    print(f"   - Max: {max_time:.2f}s")
    
    print(f"\n📋 Details:")
    for r in performance_results:
        print(f"   - {r['location']:30s} | {r['time']:5.2f}s | {r['metrics_count']:2d} metrics | {r['statistics_count']:2d} stats")



TEST: Performance: Indiranagar, Bangalore
  ✓ validate_input: validate_input: SUCCESS - Address detected, needs geocoding
  ✓ extract_intent_and_select_metrics: extract_intent_and_select_metrics: SUCCESS - Extracted intent, selected 8 metrics
  ✓ geocode_location: geocode_location: SUCCESS - Geocoded to (12.9732913, 77.6404672)
  ✓ fetch_osm_data: fetch_osm_data: SUCCESS - Fetched 25 POI categories
  ✓ calculate_statistics: calculate_statistics: SUCCESS - Calculated 8 metrics
  ✓ generate_summary: generate_summary: SUCCESS - Summary generated

✅ TEST PASSED
⏱️  Time: 4.44s

⚠️  Warnings: 2
   - Road density calculation not yet implemented
   - Road density calculation not yet implemented

TEST: Performance: Koramangala, Bangalore
  ✓ validate_input: validate_input: SUCCESS - Address detected, needs geocoding
  ✓ extract_intent_and_select_metrics: extract_intent_and_select_metrics: SUCCESS - Extracted intent, selected 8 metrics
  ✓ geocode_location: geocode_location: SUCCESS - Geocoded

## Test 4: State Inspection - Full Workflow Trace


In [11]:
test4_state = create_initial_state(
    user_input="HSR Layout, Bangalore",
    user_profile="Senior Citizen"
)

result4 = run_test(graph, test4_state, "Test 4: Full State Inspection")

if result4["success"]:
    final_state = result4["final_state"]
    
    print(f"\n{'='*60}")
    print("🔍 COMPLETE STATE INSPECTION")
    print(f"{'='*60}")
    
    print(f"\n📥 Input:")
    print(f"   - User Input: {final_state.get('user_input')}")
    print(f"   - User Profile: {final_state.get('user_profile')}")
    
    print(f"\n🌍 Geocoding:")
    print(f"   - Coordinates: {final_state.get('coordinates')}")
    print(f"   - Address: {final_state.get('address')}")
    
    print(f"\n🎯 User Intent:")
    intent = final_state.get('user_intent', {})
    pprint(intent, width=80)
    
    print(f"\n📊 Selected Metrics ({len(final_state.get('selected_metrics', []))}):")
    for metric in final_state.get('selected_metrics', []):
        print(f"   - {metric}")
    
    print(f"\n📈 Statistics ({len(final_state.get('statistics', {}))}):")
    stats = final_state.get('statistics', {})
    for key, value in list(stats.items())[:10]:
        print(f"   - {key}: {value}")
    if len(stats) > 10:
        print(f"   ... and {len(stats) - 10} more")
    
    print(f"\n📝 Summary:")
    summary = final_state.get('summary', 'N/A')
    if summary and summary != 'N/A':
        print(summary[:300] + "..." if len(summary) > 300 else summary)
    else:
        print("   No summary generated")
    
    print(f"\n📋 Processing Steps ({len(final_state.get('processing_steps', []))}):")
    for step in final_state.get('processing_steps', []):
        print(f"   - {step}")
    
    if final_state.get('errors'):
        print(f"\n❌ Errors:")
        for error in final_state.get('errors', []):
            print(f"   - {error}")
    
    if final_state.get('warnings'):
        print(f"\n⚠️  Warnings:")
        for warning in final_state.get('warnings', []):
            print(f"   - {warning}")



TEST: Test 4: Full State Inspection
  ✓ validate_input: validate_input: SUCCESS - Address detected, needs geocoding
  ✓ extract_intent_and_select_metrics: extract_intent_and_select_metrics: SUCCESS - Extracted intent, selected 7 metrics
  ✓ geocode_location: geocode_location: SUCCESS - Geocoded to (12.9116225, 77.6388622)
  ✓ fetch_osm_data: fetch_osm_data: SUCCESS - Fetched 25 POI categories
  ✓ calculate_statistics: calculate_statistics: SUCCESS - Calculated 7 metrics
  ✓ generate_summary: generate_summary: SUCCESS - Summary generated

🔍 COMPLETE STATE INSPECTION

📥 Input:
   - User Input: HSR Layout, Bangalore
   - User Profile: Senior Citizen

🌍 Geocoding:
   - Coordinates: (12.9116225, 77.6388622)
   - Address: HSR Layout, Bengaluru South City Corporation, Bengaluru, Bangalore South, Bengaluru Urban, Karnataka, India

🎯 User Intent:
{'concerns': ['accessibility', 'comfort'],
 'lifestyle': 'Comfortable, relaxed, community-based lifestyle',
 'metric_selection_reasoning': 'Selected 

## Test 5: Edge Cases


In [12]:
# Test 5a: No Profile
test5a_state = create_initial_state(
    user_input="MG Road, Bangalore",
    user_profile=None
)

result5a = run_test(graph, test5a_state, "Test 5a: No Profile (Defaults)")
print_test_results(result5a, verbose=False)

# Test 5b: Empty Input
test5b_state = create_initial_state(
    user_input="",
    user_profile="Bachelor"
)

result5b = run_test(graph, test5b_state, "Test 5b: Empty Input")
print_test_results(result5b, verbose=False)

# Test 5c: Custom Free-Text Profile
test5c_state = create_initial_state(
    user_input="Jayanagar, Bangalore",
    user_profile="I'm a fitness enthusiast who loves parks and gyms, need good connectivity"
)

result5c = run_test(graph, test5c_state, "Test 5c: Custom Free-Text Profile")
print_test_results(result5c, verbose=True)

if result5c["success"]:
    intent = result5c["final_state"].get("user_intent", {})
    selected = result5c["final_state"].get("selected_metrics", [])
    
    print(f"\n🎯 Extracted Intent:")
    print(f"   - Profile Type: {intent.get('profile_type')}")
    print(f"   - Priorities: {intent.get('priorities', [])}")
    
    print(f"\n📊 Selected Metrics:")
    fitness_metrics = [m for m in selected if 'gym' in m.lower() or 'fitness' in m.lower() or 'park' in m.lower()]
    if fitness_metrics:
        print(f"   ✅ Found fitness-related metrics: {fitness_metrics}")
    print(f"   All metrics: {selected}")



TEST: Test 5a: No Profile (Defaults)
  ✓ validate_input: validate_input: SUCCESS - Address detected, needs geocoding
  ✓ extract_intent_and_select_metrics: extract_intent_and_select_metrics: SKIPPED - No profile, used defaults
  ✓ geocode_location: geocode_location: SUCCESS - Geocoded to (12.9755264, 77.6067902)
  ✓ fetch_osm_data: fetch_osm_data: SUCCESS - Fetched 25 POI categories
  ✓ calculate_statistics: calculate_statistics: SUCCESS - Calculated 7 metrics
  ✓ generate_summary: generate_summary: SUCCESS - Summary generated

✅ TEST PASSED
⏱️  Time: 12.59s

TEST: Test 5b: Empty Input
  ✓ validate_input: validate_input: FAILED - No input provided
  ✓ handle_error: handle_error: Error handling completed

✅ TEST PASSED
⏱️  Time: 0.00s

⚠️  Errors: 2
   - User input is required
   - User input is required

TEST: Test 5c: Custom Free-Text Profile
  ✓ validate_input: validate_input: SUCCESS - Address detected, needs geocoding
  ✓ extract_intent_and_select_metrics: extract_intent_and_selec

## Summary and Conclusions


In [13]:
print(f"\n{'='*60}")
print("📋 TEST SUITE SUMMARY")
print(f"{'='*60}")

# Collect all results
all_results = []
if 'result1' in locals() and result1.get("success") is not None:
    all_results.append(result1)
if 'result2' in locals() and result2.get("success") is not None:
    all_results.append(result2)
if 'result4' in locals() and result4.get("success") is not None:
    all_results.append(result4)
if 'result5a' in locals() and result5a.get("success") is not None:
    all_results.append(result5a)
if 'result5c' in locals() and result5c.get("success") is not None:
    all_results.append(result5c)

if all_results:
    passed = sum(1 for r in all_results if r.get("success", False))
    failed = len(all_results) - passed
    
    print(f"\n✅ Passed: {passed}/{len(all_results)}")
    print(f"❌ Failed: {failed}/{len(all_results)}")
    
    if performance_results:
        avg_time = sum(r["time"] for r in performance_results) / len(performance_results)
        print(f"\n⏱️  Average Execution Time: {avg_time:.2f}s")
    
    print(f"\n🎯 Key Findings:")
    print(f"   - Parallel execution: Intent extraction runs independently")
    print(f"   - Profile handling: Works with categorical and free-text profiles")
    print(f"   - Error handling: Gracefully handles invalid inputs")
    print(f"   - Default fallback: Uses defaults when profile is missing")
    print(f"   - Metrics selection: LLM selects relevant metrics based on intent")
    
    print(f"\n{'='*60}")
else:
    print("\n⚠️  No test results available. Run the test cells above first.")



📋 TEST SUITE SUMMARY

✅ Passed: 5/5
❌ Failed: 0/5

⏱️  Average Execution Time: 17.85s

🎯 Key Findings:
   - Parallel execution: Intent extraction runs independently
   - Profile handling: Works with categorical and free-text profiles
   - Error handling: Gracefully handles invalid inputs
   - Default fallback: Uses defaults when profile is missing
   - Metrics selection: LLM selects relevant metrics based on intent

